# 시계열 예측

(출처: [Kaggle](https://www.kaggle.com/code/prashant111/complete-guide-on-time-series-analysis-in-python))

(참고 문헌: [Rob J Hyndman](https://otexts.com/fpp3/))



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# customize the style
pd.options.display.float_format = '{:.5f}'.format
pd.options.display.max_rows = 12

# load the data
df = pd.read_csv('https://raw.githubusercontent.com/selva86/datasets/master/AirPassengers.csv')
df.head()

## 시계열 데이터 탐색

In [ ]:
df = df.set_index('date')
df.index = pd.to_datetime(df.index)

In [ ]:
df.plot(style='-', figsize=(15, 5), title='Number of Air Passengers')
plt.show()

In [ ]:
# ACT, PACF
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
plot_acf(df, ax=ax1, lags=50)
plot_pacf(df, ax=ax2, lags=50)
plt.tight_layout()
plt.show()

In [ ]:
# 시계열 분해
from statsmodels.tsa.seasonal import seasonal_decompose
result = seasonal_decompose(df, model='additive', period=12)
result.plot()
plt.tight_layout()
plt.show()

In [ ]:
# 월에 따른 항공 여객 수 추이를 확인합니다
df2 = df.copy()
df2['month'] = df.index.month
fig, ax = plt.subplots(figsize=(10, 8))
sns.boxplot(data=df2, x='month', y='value')
ax.set_title('Passengers by Month')
plt.show()

## 모델링

In [ ]:
# 시계열 데이터의 train/test 분할
train_size = int(len(df) * 0.8)
train, test = df[:train_size], df[train_size:]

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

# Naive 모델
naive_forecast = test.shift(1)
naive_rmse = sqrt(mean_squared_error(test[1:], naive_forecast[1:]))
print(f"Naive 모델 RMSE: {naive_rmse}")

In [ ]:
# ARIMA 모델
from statsmodels.tsa.statespace.sarimax import SARIMAX
model = SARIMAX(train['value'], order=(1,1,1), seasonal_order=(1,1,1,12))
model_fit = model.fit()
arima_forecast = model_fit.forecast(steps=len(test))
arima_rmse = sqrt(mean_squared_error(test['value'], arima_forecast))
print(f"ARIMA 모델 RMSE: {arima_rmse}")

In [ ]:
# 딥러닝 모델 (LSTM 레이어 이용)
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

scaler = MinMaxScaler()
df['scaled_value'] = scaler.fit_transform(df[['value']])
def create_dataset(dataset, time_step=1, future_steps=1):
    X, y = [], []
    for i in range(len(dataset) - time_step - future_steps + 1):
        a = dataset[i:(i + time_step), 0]
        X.append(a)
        y.append(dataset[(i + time_step):(i + time_step + future_steps), 0])
    return np.array(X), np.array(y)

train, test = df[:train_size], df[train_size:]
time_step = 12
future_steps = 3  # 3개월 후까지 예측
X_train, y_train = create_dataset(train[['scaled_value']].values, time_step, future_steps)
X_test, y_test = create_dataset(test[['scaled_value']].values, time_step, future_steps)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

model = Sequential([
    LSTM(512, return_sequences=True, input_shape=(time_step, 1)),
    LSTM(256, return_sequences=True),
    LSTM(128, return_sequences=True),
    LSTM(64),
    Dense(future_steps)
])
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, validation_split=0.2, epochs=30, batch_size=32, verbose=1)
X_full = []
for i in range(len(df) - time_step - future_steps + 1):
    X_full.append(df['scaled_value'].values[i:(i + time_step)])
X_full = np.array(X_full).reshape(-1, time_step, 1)
full_predict = model.predict(X_full)
full_predict = scaler.inverse_transform(full_predict)
lstm_forecast = full_predict[:, -1]
lstm_forecast_series = pd.Series(lstm_forecast, index=df.index[time_step+future_steps-1:])
lstm_forecast_full = pd.Series(df['value'].values[:time_step+future_steps-1], index=df.index[:time_step+future_steps-1])
lstm_forecast_full = pd.concat([lstm_forecast_full, lstm_forecast_series])
lstm_forecast = lstm_forecast_full[test.index]
lstm_rmse = np.sqrt(mean_squared_error(test['value'], lstm_forecast))
print(f"LSTM 모델 RMSE: {lstm_rmse}")

In [ ]:
# 최종 결과 시각화
import matplotlib.pyplot as plt

test_index = test.index

naive_forecast_series = pd.Series(naive_forecast['value'].values, index=test_index)
arima_forecast_series = pd.Series(arima_forecast.values, index=test_index)
lstm_forecast_array = np.array(lstm_forecast).flatten()

lstm_index = test_index[-len(lstm_forecast_array):]

end_date = test_index[-1]
start_date = end_date - pd.Timedelta(days=3)

plt.figure(figsize=(12, 6))

plt.plot(df.index, df['value'], label='Actual value')

# Naive forecast
plt.plot(naive_forecast_series.index, naive_forecast_series, label='Naive Forecast')

# ARIMA forecast
plt.plot(arima_forecast_series.index, arima_forecast_series, label='ARIMA Forecast')

# LSTM forecast
plt.plot(lstm_index, lstm_forecast_array, label='LSTM Forecast')

plt.legend()
plt.title('Model Forecast Comparison with Actual Values')
plt.xlabel('Date')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
